<a href="https://colab.research.google.com/github/KJM94/Single_project/blob/main/%EC%9B%B9%20%EA%B4%91%EA%B3%A0%20%ED%81%B4%EB%A6%AD%EB%A5%A0%20%EC%98%88%EC%B8%A1%20AI%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/baseline_(4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import

In [ ]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install category-encoders

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import plotly.express as px

import category_encoders as ce
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
# 동일한 결과 보장을 위해 Seed값을 고정합니다
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed를 42로 고정

In [ ]:
# 제공된 train 데이터와 test 데이터를 불러옵니다
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

MemoryError: Unable to allocate 5.75 GiB for an array with shape (27, 28605391) and data type object

### EDA 1 : Sparse and Dense

In [ ]:
train.head()

### EDA 2 : Imbalance

In [ ]:
click = train['Click'].value_counts(normalize=True)

click_figure = px.bar(click,
             x=['Not Clicked : 0', 'Clicked : 1'],
             y=click.values.tolist(),
             labels={'x': 'Value', 'y': 'Percentage'},
             width = 450,
             height = 500
            )

# 그래프 표시
click_figure.show()

### Data Preprocessing 1 : Select x, y

In [ ]:
train_x = train.drop(columns=['ID', 'Click'])
train_y = train['Click']

test_x = test.drop(columns=['ID'])

### Data Preprocessing 2 : Fill NaN

In [ ]:
for col in tqdm(train_x.columns):
    if train_x[col].isnull().sum() != 0:
        train_x[col].fillna(0, inplace=True)
        test_x[col].fillna(0, inplace=True)

### Data Preprocessing 3 : Count Encoding

In [ ]:
encoding_target = list(train_x.dtypes[train_x.dtypes == "object"].index)

enc = ce.CountEncoder(cols = encoding_target).fit(train_x, train_y)
X_train_encoded = enc.transform(train_x)
X_test_encoded = enc.transform(test_x)

### Model Setting

In [ ]:
model = AdaBoostClassifier()

### Model Train and Inference

In [ ]:
model.fit(X_train_encoded, train_y)

In [ ]:
pred = model.predict_proba(X_test_encoded)
display(model.classes_)
display(pred)

### Submission

In [ ]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission

In [ ]:
sample_submission['Click'] = pred[:,1]
sample_submission

In [ ]:
sample_submission.to_csv('baseline_submission.csv', index=False)